In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from   scipy import stats
import statsmodels.formula.api as sm
import chart_studio.plotly     as py
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
import pingouin

In [3]:
df = pd.read_csv("..\DataSets\owid-covid-data.csv")
df.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2/24/2020,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2/26/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2/27/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2/28/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


## Ordenamiento de la base


In [56]:
# copia de la base de datos
copy_df = df.copy()

#convierto la coluna de fechas al formato de fecha 
copy_df['date'] = pd.to_datetime(copy_df['date'])

# Eliminar valores Nan
copy_df.replace('NaN',np.nan,inplace= True)

copy_df.fillna(0,inplace=True)

copy_df = copy_df.set_index('date')

# Agrupar por continentes
# continentes = copy_df.groupby("continent")

# Separo el contiente europeo que es con el que deseo trabaajar
df_europe = copy_df.query('continent in ["Europe"]')

df_asia = copy_df.query('continent in ["Asia"]')
# agrupo por paises la data
paises = copy_df.groupby("location")


copy_df.columns




Index(['iso_code', 'continent', 'location', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vac

# Modelo para predecir nuevos casos por pais 

In [60]:
modelE  = 'new_cases_smoothed~location + population + people_vaccinated  + people_fully_vaccinated  + hosp_patients + total_cases + reproduction_rate + total_tests + new_tests + life_expectancy + human_development_index + handwashing_facilities + gdp_per_capita + extreme_poverty'

lmE = sm.ols(formula=modelE , data= df_europe)

result = lmE.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     new_cases_smoothed   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.551
Method:                 Least Squares   F-statistic:                     1028.
Date:                Sat, 11 Feb 2023   Prob (F-statistic):               0.00
Time:                        12:00:08   Log-Likelihood:            -5.1264e+05
No. Observations:               47659   AIC:                         1.025e+06
Df Residuals:                   47601   BIC:                         1.026e+06
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [50]:
result.predict()

<bound method Results.predict of <statsmodels.regression.linear_model.OLSResults object at 0x00000170130E9410>>

# Modelo para predecir nuevas muertes por dia 


In [62]:
model2  = 'new_deaths_smoothed~location + new_cases + population + people_vaccinated  + people_fully_vaccinated  + hosp_patients + total_cases + reproduction_rate + total_tests + new_tests + life_expectancy + human_development_index + handwashing_facilities + gdp_per_capita + extreme_poverty'

lm2 = sm.ols(formula=model2 , data= copy_df)

result2 = lm2.fit()

print(result2.summary())

                             OLS Regression Results                            
Dep. Variable:     new_deaths_smoothed   R-squared:                       0.788
Model:                             OLS   Adj. R-squared:                  0.788
Method:                  Least Squares   F-statistic:                     3268.
Date:                 Sat, 11 Feb 2023   Prob (F-statistic):               0.00
Time:                         12:01:13   Log-Likelihood:            -1.5775e+06
No. Observations:               220343   AIC:                         3.155e+06
Df Residuals:                   220091   BIC:                         3.158e+06
Df Model:                          251                                         
Covariance Type:             nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [65]:
df.info

<bound method DataFrame.info of        iso_code continent     location       date  total_cases  new_cases  \
0           AFG      Asia  Afghanistan  2/24/2020          5.0        5.0   
1           AFG      Asia  Afghanistan  2/25/2020          5.0        0.0   
2           AFG      Asia  Afghanistan  2/26/2020          5.0        0.0   
3           AFG      Asia  Afghanistan  2/27/2020          5.0        0.0   
4           AFG      Asia  Afghanistan  2/28/2020          5.0        0.0   
...         ...       ...          ...        ...          ...        ...   
220338      ZWE    Africa     Zimbabwe  9/26/2022     257342.0        2.0   
220339      ZWE    Africa     Zimbabwe  9/27/2022     257376.0       34.0   
220340      ZWE    Africa     Zimbabwe  9/28/2022     257409.0       33.0   
220341      ZWE    Africa     Zimbabwe  9/29/2022     257409.0        0.0   
220342      ZWE    Africa     Zimbabwe  9/30/2022     257465.0       56.0   

        new_cases_smoothed  total_deaths  n